In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from decryption_circuit import create_decryption_circuit, create_decryption_circuit_for_deletion
from deletion_circuit import delete
from attack_circuit import breidbart_measurement
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from utils import random_bit_string
import shutil
from datetime import datetime
from experiment import Experiment
import typing

In [2]:
# Experiment parameters
experiment_id = "montreal-tamper"
execution_datetime = datetime.now()
system_string = "Fake Montreal" # Or insert other system name here
microsecond_delay = 0 # Delay between preparing the qubits and the first measurement, whether deletion or decryption
folder_prefix = "data"
folder_path = f"{folder_prefix}/{experiment_id}"

# Backend parameters
provider = IBMQ.load_account()

# Ideal simulator
# backend = AerSimulator()

# Simulator with noise model
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)

# Real backend - uncomment, might be a different way to access
# backend = provider.get_backend("ibmq_montreal")

# Qiskit parameters
optimization_level = 0
shots = 1000

In [3]:
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=optimization_level)
    transpiled_circuit = typing.cast(QuantumCircuit, transpiled_circuit)
    if draw:
        print(transpiled_circuit.draw())
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [4]:
# Encrypt 
global_params = GlobalParameters.generate_from_lambda(1)
key = Key.generate_key(global_params)
message = random_bit_string(global_params.n)
ciphertext = encrypt(message, key, global_params, microsecond_delay=microsecond_delay)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

In [5]:
# Test 1 - honest delete
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_and_measure(deletion_circuit_test1)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [6]:
# Test 2 - decrypt
decryption_circuit_test2 = create_decryption_circuit(key, ciphertext)
decryption_counts_test2 = run_and_measure(decryption_circuit_test2)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [7]:
# Test 3 - honest delete and then decrypt
deletion_circuit_test3 = delete(ciphertext)
ciphertext.circuit = deletion_circuit_test3 # To use the new circuit as the starting point of the decryption circuit
decryption_circuit_test3 = create_decryption_circuit(key, ciphertext)
raw_counts_test3 = run_and_measure(decryption_circuit_test3)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [8]:
# Test 4 - malicious delete and then decrypt
deletion_circuit_test4 = breidbart_measurement(ciphertext)
ciphertext.circuit = deletion_circuit_test4 # To use the new circuit as the starting point of the decryption circuit
decryption_circuit_test4 = create_decryption_circuit(key, ciphertext)
raw_counts_test4 = run_and_measure(decryption_circuit_test4)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [9]:
# Test 5 - tamper detection
decryption_circuit_test5 = create_decryption_circuit_for_deletion(key, ciphertext)
ciphertext.circuit = decryption_circuit_test5
deletion_circuit_test5 = delete(ciphertext)
raw_counts_test5 = run_and_measure(deletion_circuit_test5)

ciphertext.circuit = original_circuit.copy()

In [10]:
# Create experiment and export to a folder
exp = Experiment(
    experiment_id=experiment_id,
    execution_datetime=execution_datetime,
    execution_shots=shots,
    backend_system=system_string,
    microsecond_delay=microsecond_delay,
    folder_path=folder_path,
    parameters=global_params,
    key=key,
    ciphertext=ciphertext,
    message=message,
    deletion_counts_test1=deletion_counts_test1,
    decryption_counts_test2=decryption_counts_test2,
    raw_counts_test3=raw_counts_test3,
    raw_counts_test4=raw_counts_test4,
    raw_counts_test5=raw_counts_test5,
)
exp.export_to_folder()

In [11]:
# Create zip file
shutil.make_archive("data_export", "zip", folder_prefix);